## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column.
attrition_df.nunique()

,0
Age,43
Attrition,2
BusinessTravel,3
Department,3
DistanceFromHome,29
Education,5
EducationField,6
EnvironmentSatisfaction,4
HourlyRate,71
JobInvolvement,4


In [3]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]

In [4]:
# Create a list of at least 10 column names to use as X data
X = ['Education', 'Age', 'DistanceFromHome', 'JobSatisfaction', 'OverTime', 'StockOptionLevel', 'WorkLifeBalance', 'YearsAtCompany', 'YearsSinceLastPromotion', 'NumCompaniesWorked']

# Create X_df using your selected columns
X_df = attrition_df[X]

# Show the data types for X_df
X_df.dtypes

,0
Education,int64
Age,int64
DistanceFromHome,int64
JobSatisfaction,int64
OverTime,object
StockOptionLevel,int64
WorkLifeBalance,int64
YearsAtCompany,int64
YearsSinceLastPromotion,int64
NumCompaniesWorked,int64


In [5]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
print(X_df['OverTime'].value_counts())
X_df['OverTime'] = X_df['OverTime'].apply(lambda x: 1 if x == 'Yes' else 0)
# X_df.loc[:,'OverTime'] = X_df['OverTime'].replace({'Yes': 1, 'No': 0}).astype(int) (alternative way)

OverTime
No     1054
Yes     416
Name: count, dtype: int64


<ipython-input-5-2b703e5d1fa9>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_df['OverTime'] = X_df['OverTime'].apply(lambda x: 1 if x == 'Yes' else 0)


In [6]:
# Checking dtype update
X_df.dtypes

,0
Education,int64
Age,int64
DistanceFromHome,int64
JobSatisfaction,int64
OverTime,int64
StockOptionLevel,int64
WorkLifeBalance,int64
YearsAtCompany,int64
YearsSinceLastPromotion,int64
NumCompaniesWorked,int64


In [7]:
# Split the data into training and testing sets --- Moved this cell below coversion cell for better flow (splitting after converting all of X_df Overtime column to numeric)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=1)

In [8]:
# Create a StandardScaler
import sklearn
X_scaler = sklearn.preprocessing.StandardScaler()

# Fit the StandardScaler to the training data
X_scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
# Create a OneHotEncoder for the Department column
department_encoder = OneHotEncoder(sparse_output = False)

# Fit the encoder to the training data
department_encoder.fit(np.array(y_train['Department']).reshape(-1,1))

# Create two new variables by applying the encoder
# to the training and testing data
department_train_encoded = department_encoder.transform(np.array(y_train['Department']).reshape(-1,1))
department_test_encoded = department_encoder.transform(np.array(y_test['Department']).reshape(-1,1))
department_train_encoded

array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       ...,
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [10]:
# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
attrition_encoder.fit(np.array(y_train['Attrition']).reshape(-1,1))

# Create two new variables by applying the encoder
# to the training and testing data
attrition_train_encoded = attrition_encoder.transform(np.array(y_train['Attrition']).reshape(-1,1))
attrition_test_encoded = attrition_encoder.transform(np.array(y_test['Attrition']).reshape(-1,1))
attrition_train_encoded

array([[1., 0.],
       [1., 0.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [11]:
# Find the number of columns in the X training data
number_input_features = X_train_scaled.shape[1]

# Create the input layer
input_layer = layers.Input(shape=(number_input_features,))

# Create at least two shared layers
hidden_layer_1 = layers.Dense(units=10, activation="relu")(input_layer)
hidden_layer_2 = layers.Dense(units=6, activation="relu")(hidden_layer_1)

In [12]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden = layers.Dense(units=8, activation='relu')(hidden_layer_2)

# Create the output layer
department_output = layers.Dense(department_train_encoded.shape[1], activation='sigmoid', name='department')(department_hidden)

In [13]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden = layers.Dense(units=8, activation='relu')(hidden_layer_2)

# Create the output layer
attrition_output = layers.Dense(attrition_train_encoded.shape[1], activation='sigmoid', name='attrition')(attrition_hidden)

In [14]:
# Create the model
model = Model(inputs=input_layer, outputs=[department_output, attrition_output])

# Compile the model
model.compile(optimizer='adam',
              loss={'department': 'categorical_crossentropy',
                    'attrition': 'binary_crossentropy'},
              metrics={'department': 'accuracy', 'attrition': 'accuracy'})

# Summarize the model
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 10)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 10)             │            110 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 6)              │             66 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 8)              │             56 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)           │ (None, 8)              │             56 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department (Dense)        │ (None, 3)              │             27 │ dense_2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition (Dense)         │ (None, 2)              │             18 │ dense_3[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 333 (1.30 KB)

 Trainable params: 333 (1.30 KB)

 Non-trainable params: 0 (0.00 B)

In [15]:
# Train the model
model.fit(
    X_train_scaled,
    {'department': department_train_encoded,
     'attrition': attrition_train_encoded},
    epochs=100
    #batch_size=64,
    #validation_split=0.3,
    #shuffle=True
)

Epoch 1/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - attrition_accuracy: 0.8564 - department_accuracy: 0.6648 - loss: 1.5734
Epoch 2/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_accuracy: 0.8314 - department_accuracy: 0.6644 - loss: 1.4697
Epoch 3/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_accuracy: 0.8525 - department_accuracy: 0.6530 - loss: 1.3759
Epoch 4/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_accuracy: 0.8477 - department_accuracy: 0.6537 - loss: 1.3037
Epoch 5/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_accuracy: 0.8557 - department_accuracy: 0.6601 - loss: 1.2523
Epoch 6/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - attrition_accuracy: 0.8435 - department_accuracy: 0.6536 - loss: 1.2270
Epoch 7/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_accuracy: 0.8302 - department_accuracy: 0.6582 - loss: 1.2227
Epoch 8/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_accuracy: 0.8511 - department_accuracy: 0.6671 - loss: 1.1924


In [16]:
# Evaluate the model with the testing data
test_results = model.evaluate(X_test_scaled, {'department': department_test_encoded, 'attrition': attrition_test_encoded})
test_results

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_accuracy: 0.8283 - department_accuracy: 0.6296 - loss: 1.2645  


[1.1973625421524048, 0.8478260636329651, 0.6413043737411499]

In [17]:
# Print the accuracy for both department and attrition
print(f"Department Predictions Accuracy: {test_results[1]}")
print(f"Attrition Predictions Accuracy: {test_results[2]}")

Department Predictions Accuracy: 0.8478260636329651
Attrition Predictions Accuracy: 0.6413043737411499


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Accuracy is a good method to use on this data because it gives us a realistic measure of how the model is performing when predicting department and attrition. Having the model loss is also helpful. As an alternative or better yet, supplement, we could have calculated the msre as well.
2. The activation function I chose for the output layers was sigmoid, because the results for a subject may be independent from one another and could belong to multiple classes. However, you can also argue for the softmax function because the results may also be mutually exclusive if a subject is classified as belonging to a single department for example. Furthermore, I used relu for the hidden layers because it is considered the default activation function, its easier/faster to train and usually achieves a better performance.
3. In order to improve the model, I could have played around with the activation functions, number of epochs, added more layers, etc. I could have further tested and played around with all these different things until I optimized the accuracy of the model.